In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [40]:
train = pd.read_csv ('/Users/prajjwal/Library/CloudStorage/OneDrive-UniversityofAppliedSciencesEuropeGmbH(ehem.BiTSbtk)-Berlin,Hamburg,Iserlohn/Kaggel/bike-sharing-demand/train.csv')
test = pd.read_csv ('/Users/prajjwal/Library/CloudStorage/OneDrive-UniversityofAppliedSciencesEuropeGmbH(ehem.BiTSbtk)-Berlin,Hamburg,Iserlohn/Kaggel/bike-sharing-demand/test.csv')

print(train.head())
print(train.info())
print(test.head())
print(test.info())

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-n

In [42]:
print(train.isnull().sum())
print(test.isnull().sum())

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64
datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64


In [48]:
test['datetime'] = pd.to_datetime(test['datetime'])
train['datetime'] = pd.to_datetime(train['datetime'])

for df in [test,train]:
 df['year'] = df['datetime'].dt.year
 df['month'] = df['datetime'].dt.month
 df['weekday'] = df['datetime'].dt.weekday
 df['day'] = df['datetime'].dt.day
 df['hour'] = df['datetime'].dt.hour

print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int32         
 13  month       10886 non-null  int32         
 14  weekday     10886 non-null  int32         
 15  day         10886 non-null  int32         
 16  hour        10886 non-

In [50]:
hourly_weather_stats = train.groupby(['hour', 'weather'])['count'].mean().reset_index()
hourly_weather_stats.rename(columns={'count': 'avg_hourly_weather_count'}, inplace=True)

train = train.merge(hourly_weather_stats, on=['hour', 'weather'], how='left')
test = test.merge(hourly_weather_stats, on=['hour', 'weather'], how='left')

In [52]:
# Drop irrelevant columns for training
X_train = train.drop(['datetime', 'casual', 'registered', 'count'], axis=1)
y_train = train['count']

# Prepare test features
X_test = test.drop(['datetime'], axis=1)  # Keep datetime for final submission

In [86]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

# Create a SimpleImputer instance (mean strategy is used here)
imputer = SimpleImputer(strategy='mean')
X_test['avg_hourly_weather_count'] = imputer.fit_transform(X_test[['avg_hourly_weather_count']])

# Initialize the model
model = RandomForestRegressor(n_estimators=500, random_state=42)

# Create a pipeline that first imputes the data, then fits the Random Forest model
pipeline = make_pipeline(imputer, model)

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict using the trained model
y_pred = pipeline.predict(X_test)
rmse = mean_squared_error(y_train, y_train_pred, squared=False)
print(f"Training RMSE: {rmse}")

Training RMSE: 15.188728821556175


In [88]:
print(pd.DataFrame(X_test).isnull().sum())

season                      0
holiday                     0
workingday                  0
weather                     0
temp                        0
atemp                       0
humidity                    0
windspeed                   0
year                        0
month                       0
weekday                     0
day                         0
hour                        0
avg_hourly_weather_count    0
dtype: int64


In [90]:
# Predict on test data
test['count'] = model.predict(X_test)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [92]:
# Create submission DataFrame
submission = test[['datetime', 'count']]

# Save to CSV
submission.to_csv('submission.csv', index=False)